In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

essays_train = pd.DataFrame(pd.read_csv('/Users/aryan/Actual-Coding/CDAC/feedback-prize-english-language-learning/train.csv'))
essays = essays_train['full_text']
train_texts = essays[1:2].to_list()
coh_train = essays_train['cohesion']
train_labels = coh_train[1:2].to_list()

In [2]:
from transformers import BartTokenizer

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-mnli")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

In [3]:
train_encodings

{'input_ids': [[0, 1779, 10, 936, 16, 10, 464, 47, 33, 7, 905, 24, 109, 5, 275, 15, 47, 117, 948, 99, 16, 2909, 24, 64, 464, 110, 1508, 4, 2128, 47, 240, 7, 3874, 62, 8, 356, 99, 16, 198, 47, 142, 1272, 32, 5, 275, 169, 7, 464, 99, 47, 236, 7, 464, 552, 86, 536, 4, 83, 50118, 50118, 41045, 16, 10, 464, 13, 47, 142, 24, 64, 146, 47, 192, 430, 8, 244, 47, 7, 1346, 141, 326, 1033, 885, 1638, 4, 50118, 50118, 10993, 9, 70, 24, 64, 146, 47, 192, 430, 172, 5, 643, 4, 286, 1246, 939, 2145, 14, 77, 939, 376, 7, 5, 315, 532, 939, 206, 14, 1085, 21, 164, 7, 464, 162, 142, 939, 206, 14, 1085, 21, 164, 7, 464, 162, 142, 960, 21, 430, 14, 127, 247, 8, 172, 939, 588, 661, 14, 1593, 142, 10, 936, 189, 464, 47, 53, 2128, 64, 45, 464, 5, 169, 24, 16, 6, 53, 939, 2145, 14, 939, 21, 269, 9152, 53, 939, 206, 14, 464, 10, 319, 142, 2128, 127, 1272, 146, 162, 206, 14, 89, 16, 55, 631, 14, 939, 393, 192, 11, 127, 301, 53, 939, 95, 240, 7, 192, 24, 31, 10, 430, 169, 8, 33976, 905, 1085, 1102, 8, 910, 12448, 5

In [4]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

In [5]:
from transformers import TFBartForSequenceClassification

model = TFBartForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.


In [6]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True
)

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.2, 
    verbose = 1, 
    patience = 5,                        
    min_lr = 0.001 # type: ignore
)

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy']) # type: ignore
model.fit(train_dataset.shuffle(1).batch(1), epochs=3, verbose = 1) # type: ignore

Epoch 1/3


In [ ]:
model.save_pretrained("/Users/aryan/Actual-Coding/CDAC/feedback-prize-english-language-learning") # type: ignore
tokenizer.save_pretrained("/Users/aryan/Actual-Coding/CDAC/feedback-prize-english-language-learning")

In [ ]:
model = TFBartForSequenceClassification.from_pretrained("/Users/aryan/Actual-Coding/CDAC/feedback-prize-english-language-learning")
tokenizer = BartTokenizer.from_pretrained("/Users/aryan/Actual-Coding/CDAC/feedback-prize-english-language-learning")